In [57]:
import pandas as pd
from datetime import date
import os


def get_filepaths(directory):
    '''
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    '''
    file_names = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            file_names.append(filename)  # Add it to the list.

    return file_names  # Output


def check_update_years(files):
    '''
    
    '''
    current_year = date.today().year # get current year
    update_years = [str(year) for year in range(2010, current_year)] # opendatabcn-accidents available years
    
    # Check years present in topic's current landing zone
    present_years = [element[:4] for element in files] # returns a list of present years
    
    #
    for element in present_years:
        if element in update_years:
            update_years.remove(element)
    
    return update_years


def collect_data(update_years):
    '''
    
    '''
    for year in update_years:
        try:
            df = pd.read_csv('https://opendata-ajuntament.barcelona.cat/data/dataset/e769eb9d-d778-4cd7-9e3a-5858bba49b20/resource/bcfc0866-7e2a-4054-9a93-fb3f371fc5eb/download/{}_accidents_gu_bcn.csv'.format(year))
            df.to_csv('landing/opendatabcn-accidents/{}_accidents_bcn.csv'.format(year), encoding='utf-8')
        except:
            print('unable to extract accidents data for {}'.format(year))

            
if __name__ == __main__:
    #
    opendatabcn_filenames = get_filepaths("landing/temporal/opendatabcn-accidents")
    update_years = check_update_years(opendatabcn_filenames)
    
    # opendatabcn data collection
    if update_years:
        collect_data(update_years)
        print('update complete!')
    else:
        print('accidents data is already up to date!')

NameError: name '__main__' is not defined

In [31]:
if __name__ == __main__:
    # Run the above function and store its results in a variable.   
    opendatabcn_filenames = get_filepaths("landing/temporal/opendatabcn-accidents")
    update_years = check_update_years(opendatabcn_filenames)
    
    # opendatabcn data collection
    if update_years:
        collect_data(update_years)
        print('update complete!')
    else:
        print('accidents data is already up to date!')

In [56]:
accidents_years = [str(year) for year in range(2010, current_year)] # opendatabcn-accidents available years
present_years = [element[:4] for element in opendatabcn_filenames] # returns a list of present years

for element in present_years:
    if element in accidents_years:
        accidents_years.remove(element)

print(accidents_years)

['2009']


In [48]:
present_years

['2015',
 '2021',
 '2010',
 '2014',
 '2020',
 '2011',
 '2017',
 '2012',
 '2016',
 '2013',
 '2018',
 '2019']

In [49]:
accidents_years = [str(year) for year in range(2010, current_year)] # opendatabcn-accidents available years
accidents_years

['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021']

In [59]:
df = pd.read_csv('https://opendata-ajuntament.barcelona.cat/data/dataset/e769eb9d-d778-4cd7-9e3a-5858bba49b20/resource/bcfc0866-7e2a-4054-9a93-fb3f371fc5eb/download/2021_accidents_gu_bcn.csv')





In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6268 entries, 0 to 6267
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Numero_expedient           6268 non-null   object 
 1   Codi_districte             6268 non-null   int64  
 2   Nom_districte              6268 non-null   object 
 3   Codi_barri                 6268 non-null   object 
 4   Nom_barri                  6268 non-null   object 
 5   Codi_carrer                6268 non-null   int64  
 6   Nom_carrer                 6268 non-null   object 
 7   Num_postal_caption         6268 non-null   object 
 8   Descripcio_dia_setmana     6268 non-null   object 
 9   Dia_setmana                6268 non-null   int64  
 10  Descripcio_tipus_dia       6268 non-null   object 
 11  NK_Any                     6268 non-null   int64  
 12  Mes_any                    6268 non-null   int64  
 13  Nom_mes                    6268 non-null   objec